In [10]:
from bs4 import BeautifulSoup
import requests as rq 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.compat import StringIO as io
from pandas.io.json import json_normalize
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import numpy as np
#pd.set_option('display.max_rows', None)
import os
import dill
import json

# Second Data Pull: Capital & Major Cities 

## Learning Moment

### The quality of your data source is vital
- Insider knowledge helps you determine this. I know that the other major South African cities listed in this source are legit because I've lived here. 

### Check how things are spelt
- Having DR Congo and The Democratic Republic of Congo is a problem when merging

In [11]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
afrimain = 'http://www.hostels247.com/hostel_247blog_Countries__in__Africa/' 
afrimain_store = rq.get(afrimain, headers=header)
afrimain_soup = BeautifulSoup(afrimain_store.text, 'html.parser')
print(afrimain_soup.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="List of the Countries in Africa, the Capital cities and major cities in Africa, Travel to Africa, Africa Hostels and Budget Hotels" name="keywords"/>
  <meta content="Hostels247.com List of Countries in Africa, the Capital cities and major cities in Africa from North, East, South and West Africa. Travel to Africa." name="description"/>
  <meta content="index,follow,all" name="robots"/>
  <meta content="index,follow" name="GOOGLEBOT"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="GENERAL" name="RATING"/>
  <meta content="2 DAYS" name="REVISIT-AFTER"/>
  <meta content="DOCUMENT" name="RESOURCE-TYPE"/>
  <meta content="GLOBAL" name="DISTRIBUTION"/>
  <title>
   List of the Countries in Africa, the Capital cities and major cities in Africa - Hostels247.com - Travel to Africa
  </title>
  <link .ico"="" href="/favicon.ico" rel="icon"/>
  <link hre

In [12]:
afrimain_soup_1 = afrimain_soup.find('td', class_='search_txt')
print(afrimain_soup_1.prettify())

<td align="left" class="search_txt" valign="top" width="55%">
 <strong>
  Countries in Africa
 </strong>
 <br/>
 <p>
  <strong>
   <font color="#00ccff">
    List of the Countries in Africa,
   </font>
   <font color="#ffcc00">
    the Capital cities
   </font>
   <font color="#0000ff">
    and major cities in Africa.
   </font>
  </strong>
  <br/>
  <br/>
  All of these African Countries and Cities are top Business, Holiday, Travel and Cultural destinations.
  <br/>
  <br/>
  <div align="center">
   <font color="#0000ff">
    <strong>
     To Book Hostels &amp; Budget Hotels click the Button below.
    </strong>
   </font>
   <a href="javascript:void(0);/*1225736966037*/">
    <font color="#0000ff">
     <br/>
    </font>
   </a>
  </div>
  <p align="center">
   <a href="http://www.hostels247.com/">
    <img align="left" alt="" height="26" src="/userfiles/image/Book%20Now%20Button.png" width="119"/>
   </a>
   Africa has many Traditions and Cultures unique to each country and city, th

In [13]:
afrimain_soup_1.find_all('a')[-1].decompose()

In [14]:
for links in afrimain_soup_1.find_all('a'):
    links.unwrap()

In [15]:
print(afrimain_soup_1.prettify())

<td align="left" class="search_txt" valign="top" width="55%">
 <strong>
  Countries in Africa
 </strong>
 <br/>
 <p>
  <strong>
   <font color="#00ccff">
    List of the Countries in Africa,
   </font>
   <font color="#ffcc00">
    the Capital cities
   </font>
   <font color="#0000ff">
    and major cities in Africa.
   </font>
  </strong>
  <br/>
  <br/>
  All of these African Countries and Cities are top Business, Holiday, Travel and Cultural destinations.
  <br/>
  <br/>
  <div align="center">
   <font color="#0000ff">
    <strong>
     To Book Hostels &amp; Budget Hotels click the Button below.
    </strong>
   </font>
   <font color="#0000ff">
    <br/>
   </font>
  </div>
  <p align="center">
   <img align="left" alt="" height="26" src="/userfiles/image/Book%20Now%20Button.png" width="119"/>
   Africa has many Traditions and Cultures unique to each country and city, the people of Africa speak Hundreds of Languages and any visitor to Africa can expect a warm welcome no matter whi

## Note: Replace Tag Content:

afrimain_soup_1.find('strong').contents[0].replaceWith(afrimain_soup_1.find('strong').text+' Brexit')

In [16]:
for fonts in afrimain_soup_1.find_all('font'):
    fonts.decompose()

In [17]:
print(afrimain_soup_1.prettify())

<td align="left" class="search_txt" valign="top" width="55%">
 <strong>
  Countries in Africa
 </strong>
 <br/>
 <p>
  <strong>
  </strong>
  <br/>
  <br/>
  All of these African Countries and Cities are top Business, Holiday, Travel and Cultural destinations.
  <br/>
  <br/>
  <div align="center">
  </div>
  <p align="center">
   <img align="left" alt="" height="26" src="/userfiles/image/Book%20Now%20Button.png" width="119"/>
   Africa has many Traditions and Cultures unique to each country and city, the people of Africa speak Hundreds of Languages and any visitor to Africa can expect a warm welcome no matter which country or city they choose to visit.
   <br/>
   <br/>
   Africa has a lot to offer from, North, South, East and West. From the well known tourist and business destinations like, Nigeria, the Heart Of Africa, the most populous country in Africa, Egypt, South Africa, Morocco, Tunisia, Mauritius and Kenya, to the hidden treasures that be found in every African country and ci

In [18]:
for strongs in afrimain_soup_1.find_all('strong'):
    strongs.decompose()

In [19]:
afrimain_list = afrimain_soup_1.text.split('\n\r\n')
#afrimain_list

### Fix: Need to run this cell multiple times 

In [20]:
for x, text in enumerate(afrimain_list):
    if 'Capital City' not in text:
        del(afrimain_list[x])
    else:
        pass
#afrimain_list

In [21]:
new_list = []

for x, cities in enumerate(afrimain_list):
    if ':' in cities:     
        new_list.append(afrimain_list[x].split(':'))
    else:
        new_list.append(afrimain_list[x].split('  '))

#new_list

In [22]:
phrase = 'Capital City in '
for x in range(len(new_list)):
    if phrase in new_list[x][0]: 
        new_list[x][0] = new_list[x][0].replace(phrase,'')
    else:
        new_list[x][0] = new_list[x][0].replace('Capital City ','')
#new_list

In [23]:
phrase_2 = '\r\nOther major cities in'
for x in range(len(new_list)):
    if new_list[x][0]== 'Cote D’Ivoire':
        new_list[x][1] = new_list[x][1].replace(phrase_2 + ' Cote D’lvoire', phrase_2 + ' Cote D’Ivoire')
    new_list[x][1] = new_list[x][1].replace(phrase_2 + ' ' + new_list[x][0],'')
#new_list

IndexError: list index out of range

In [ ]:
afristring_list = []

for d in range(len(new_list)):
    afristring_list.extend([new_list[d][0], new_list[d][1]]) 
#afristring_list

In [ ]:
afristring = '\n\n'.join(afristring_list)
#afristring

In [ ]:
afristring0 = afristring.replace('.','')
afristring1 = afristring0.replace(',','\n,')
afristring2 = afristring1.replace('\n\n ',',')
afristring3 = afristring2.replace('\n\n','\n')
afristring4 = afristring3.replace(' \n','\n')
afristring5 = afristring4.replace(', ',',')
afristring5

In [ ]:
afristring5 = afristring5[:-1]
afristring5 = afristring5.replace('Congo\n,Rep of','Republic of the Congo') 
afristring5 = afristring5.replace('Congo\n,Dem Rep of','Democratic Republic of the Congo')
afristring5

In [ ]:
afrimain_df1 = pd.read_csv(io(afristring5), names=['Country','City'])

In [ ]:
#afrimain_df1

In [ ]:
afrimain_df2=afrimain_df1[afrimain_df1['City'].notnull()]
#afrimain_df2

In [ ]:
afrimain_df2.reset_index(drop=True, inplace=True)

In [ ]:
afrimain_df2['Type']= np.nan

In [ ]:
for i in range(281): 
    #print(i)
    try:
     if np.isnan(afrimain_df2['Country'][i]):
        afrimain_df2['Type'][i] = 'MC'
    except:
        afrimain_df2['Type'][i] = 'CC'

In [ ]:
afrimain_df2.head()

In [ ]:
afrimain_df2.fillna(method='ffill',inplace=True)
afrimain_df2

In [ ]:
#afrimain_df2
afrimain_df2['Latitude']=None 
afrimain_df2['Longitude']=None

In [ ]:
geolocator = Nominatim(user_agent="cities",timeout=5) #To avoid HTTP 403 Forbidden error 

for city, country, row in zip(afrimain_df2['City'],afrimain_df2['Country'],afrimain_df2.index): 

    location = None
    i = 0

    while(location is None and i<6):

        location = geolocator.geocode(city+', '+country)

        print(city+', '+country)
        
        i+=1

    try:

        afrimain_df2['Latitude'][row]=location.latitude

        afrimain_df2['Longitude'][row]=location.longitude

    except:
        
        print('Failed: '+city+' '+country)

        afrimain_df2['Latitude'][row] = None

        afrimain_df2['Longitude'][row] = None

print("Done")

In [ ]:
afrimain_df2.head()

In [ ]:
afrimain_df2[afrimain_df2['Longitude'].isnull()]

In [ ]:
location1 = geolocator.geocode('Addis Ababa, Ethiopia')

afrimain_df2['Latitude'][100]=location1.latitude

afrimain_df2['Longitude'][100]=location1.longitude

In [ ]:
location2 = geolocator.geocode('Algiers, Algeria')

In [ ]:
location2.longitude

In [ ]:
#Dataframe with no nulls 
afrimain_df3 = afrimain_df2[afrimain_df2['Longitude'].notnull()]

In [ ]:
afrimain_df3.reset_index(drop=True,inplace=True)

In [ ]:
afrimain_df3.to_csv('City Coordinates')

In [ ]:
afrimain_caps = afrimain_df3[afrimain_df3['Type']=='CC']

In [ ]:
afrimain_caps.reset_index(drop=True,inplace=True)

In [ ]:
#afrimain_caps

### Foursquare Credentials

In [ ]:
CLIENT_ID = 'EQ0IE0NJHFCKD5SXZX5BMLBW5ATD4ZWDO5FUCRGD3QCSBBHZ' # your Foursquare ID
CLIENT_SECRET = '01KHK4CNABQSQOI2VQL5J2ATJMO5IIYIEKV3BKQMW2XAOOEJ' # your Foursquare Secret
VERSION = '20190107' # Foursquare API version

In [ ]:
afrimain_df2[afrimain_df2['Country']=='Algeria']

In [ ]:
city_lat =  36.7
city_long = 3.1

city_name = 'Algiers'

In [ ]:
Limit = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    city_lat,
    city_long,
    Limit)

In [ ]:
results1 = rq.get(url).json()

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results1['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

In [ ]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [ ]:
nearby_venues.head()

In [ ]:
#dir_path = os.path.dirname(os.path.realpath(__file__))
#os.getcwd()
os.chdir('/Users/mbongeni/Documents/Career/Data Science/Applied Data Science/4_Capstone/Final Assignment')

In [ ]:
results1

In [ ]:
results1['response'].keys()

In [51]:
with open('AfricaTours_lat&long.pkl', 'rb') as file:
    afrimain_df3 = dill.load(file)

In [ ]:
def getCityVenues(countries, cities, city_categories, latitudes, longitudes):
    
    LIMIT = 100
    
    venues_list=[]
    for country, city, cat, lat, lng in zip(countries, cities, city_categories, latitudes, longitudes):
        print(city)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,  
            LIMIT)
        
        # make the GET request
        feedback = rq.get(url).json()
        #print(feedback['response'].keys())
        total_results = feedback['response']['totalResults'] 
        places = feedback['response']['groups'][0]['items']
        
        #Filter countries with too few venues 
        if total_results >= 30:
            print('qualifies')
        
            # return only relevant information for each nearby venue
            venues_list.append([(
                    country, 
                    city,
                    cat,
                    lat, 
                    lng, 
                    v['venue']['name'], 
                    #v['venue']['location']['lat'], 
                    #v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name']) for v in places])
        else:
            print('does not qualify')

    top_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    top_venues.columns = ['Country', 
                  'City', 
                  'Type', 
                  'Latitude', 
                  'Longitude', 
                  'Venue Name', 
                  'Venue Category']
    
    return(top_venues)

In [ ]:
#Correction: Algiers Coordinates 
afrimain_df3['Latitude'][0]= 36.7
afrimain_df3['Longitude'][0]= 3.15

In [ ]:
#getCityVenues(afrimain_df3['Country'][0:12],afrimain_df3['City'][0:12],afrimain_df3['Type'][0:12],afrimain_df3['Latitude'][0:12],afrimain_df3['Longitude'][0:12])

#city_venues = getCityVenues(afrimain_df3['Country'],afrimain_df3['City'],afrimain_df3['Type'],afrimain_df3['Latitude'],afrimain_df3['Longitude'])

In [ ]:
city_venues.shape

In [ ]:
len(city_venues['Country'].unique())

In [ ]:
len(city_venues['Venue Category'].unique())

In [ ]:
len(city_venues['City'].unique())

In [ ]:
#Save Variable city_venues 

#with open('AfricaTours_venues.pkl', 'wb') as file:
    #dill.dump(city_venues, file)

In [9]:
#Load city_venues variable

with open('AfricaTours_venues.pkl', 'rb') as file:
    city_venues = dill.load(file)

In [31]:
city_venues.head(200)

,Country,City,Type,Latitude,Longitude,Venue Name,Venue Category
0,Algeria,Algiers,CC,36.700000,3.150000,Casbah İstanbul,Turkish Restaurant
1,Algeria,Algiers,CC,36.700000,3.150000,Best Night Hotel,Hotel
2,Algeria,Algiers,CC,36.700000,3.150000,Salad box Algerie,Salad Place
3,Algeria,Algiers,CC,36.700000,3.150000,Jardin d'essais d'El Hamma,Park
4,Algeria,Algiers,CC,36.700000,3.150000,The Crystal Lounge,Restaurant
5,Algeria,Algiers,CC,36.700000,3.150000,Taste Of India,Indian Restaurant
6,Algeria,Algiers,CC,36.700000,3.150000,Le Tantra,French Restaurant
7,Algeria,Algiers,CC,36.700000,3.150000,LEONARD,Restaurant
8,Algeria,Algiers,CC,36.700000,3.150000,Piano Piano,Lounge
9,Algeria,Algiers,CC,36.700000,3.150000,Cosmopolitain,Bar


## Analaysis

In [41]:
# one hot encoding
city_venues_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

#add country & city columns back to dataframe

city_venues_onehot = city_venues[['Country','City']].join( city_venues_onehot)

In [42]:
city_venues_onehot.head(30)

,Country,City,Accessories Store,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Campground,Casino,Castle,Caucasian Restaurant,Chinese Restaurant,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cricket Ground,Cultural Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dive Spot,Donut Shop,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Heliport,Herbs & Spices Store,Himalayan Restaurant,Historic Site,History Museum,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Language School,Latin American Restaurant,Lebanese Restaurant,Library,Lighthouse,Liquor Store,Lounge,Luggage Store,Market,Massage Studio,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Mongolian Restaurant,Moroccan Restaurant,Mosque,Motel,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Office,Opera House,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pharmacy,Photography Studio,Piano Bar,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Racetrack,Recreation Center,Resort,Rest Area,Restaurant,Roof Deck,Rugby Pitch,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shop & Service,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soccer Stadium,Social Club,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sports Bar,Sports Club,Stables,Stadium,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Swiss Restaurant,Syrian Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Tiki Bar,Toll Plaza,Trail,Train Station,Travel & Transport,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volcano,Water Park,Waterfront,Whisky Bar,Wings Joint,Zoo,Zoo Exhibit
0,Algeria,Algiers,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [43]:
city_venues_grouped = city_venues_onehot.groupby('City', sort=False).mean().reset_index()

In [44]:
city_venues_grouped.shape

(57, 268)

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
city_venues_sorted = pd.DataFrame(columns=columns)
city_venues_sorted['City'] =city_venues_grouped['City']

for ind in np.arange(city_venues_grouped.shape[0]):
    city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(city_venues_grouped.iloc[ind, :], num_top_venues)

city_venues_sorted

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Algiers,Hotel,Restaurant,Mediterranean Restaurant,Coffee Shop,French Restaurant,Park,Cupcake Shop,Burger Joint,Salad Place,Recreation Center,Plaza,Convenience Store,Indian Restaurant,Department Store,Steakhouse,Lounge,Lighthouse,Lake,Italian Restaurant,Gym / Fitness Center
1,Oran,Hotel,Restaurant,Café,Ice Cream Shop,Boat or Ferry,Sandwich Place,Breakfast Spot,Shopping Mall,Burger Joint,Seafood Restaurant,Scenic Lookout,Indian Restaurant,Tea Room,Eastern European Restaurant,Pizza Place,Construction & Landscaping,Dessert Shop,Pedestrian Plaza,Big Box Store,Middle Eastern Restaurant
2,Bab Ezzouar,Hotel,Mediterranean Restaurant,Coffee Shop,Restaurant,Lake,Plaza,Recreation Center,Salad Place,Burger Joint,Shopping Mall,Metro Station,Park,French Restaurant,Lounge,Steakhouse,Beach,Indian Restaurant,Cupcake Shop,Airport Terminal,Turkish Restaurant
3,Luanda,Restaurant,Pizza Place,African Restaurant,Lounge,Hotel,Seafood Restaurant,Portuguese Restaurant,Ice Cream Shop,Italian Restaurant,Bar,Mediterranean Restaurant,Beach,BBQ Joint,Coffee Shop,Convenience Store,Supermarket,Fast Food Restaurant,Dessert Shop,Office,Movie Theater
4,Cotonou,Resort,Hotel,Restaurant,Shopping Mall,Hotel Bar,Bakery,Plaza,Middle Eastern Restaurant,Beach,Lounge,Music Venue,Tapas Restaurant,French Restaurant,Thai Restaurant,Food Court,Pizza Place,Soccer Stadium,Dessert Shop,Harbor / Marina,Hardware Store
5,Gaborone,Shopping Mall,Hotel,Restaurant,Coffee Shop,Café,Burger Joint,Portuguese Restaurant,Fast Food Restaurant,Bistro,Steakhouse,Miscellaneous Shop,Cocktail Bar,Middle Eastern Restaurant,Seafood Restaurant,Brazilian Restaurant,Shop & Service,Soccer Field,Mexican Restaurant,Food,Hotel Bar
6,Yaounde,Bakery,Hotel,Plaza,Bar,Restaurant,Pizza Place,Lounge,Nightclub,Shopping Mall,Café,Food & Drink Shop,Fast Food Restaurant,Bus Station,Pub,Casino,Tennis Court,Cocktail Bar,Coffee Shop,Comedy Club,Piano Bar
7,Douala,Bakery,Lounge,Restaurant,Hotel,Shopping Mall,Nightclub,Big Box Store,Diner,Pizza Place,French Restaurant,Ice Cream Shop,African Restaurant,Department Store,Food Court,Mediterranean Restaurant,Bed & Breakfast,Pharmacy,Plaza,Pub,Café
8,Santa Maria,Resort,Hotel,Seafood Restaurant,Restaurant,Hotel Bar,Beach,Beach Bar,Surf Spot,Italian Restaurant,Spa,African Restaurant,Lounge,Portuguese Restaurant,Bistro,Steakhouse,Cocktail Bar,Botanical Garden,English Restaurant,Pub,BBQ Joint
9,Kinshasa,Hotel,Café,Restaurant,Fast Food Restaurant,Lounge,Plaza,Resort,Furniture / Home Store,Market,Steakhouse,Cocktail Bar,Grocery Store,Pool,Bar,Italian Restaurant,Diner,Pizza Place,Shopping Mall,Breakfast Spot,Airport


In [52]:
countries_df = afrimain_df3[['Country','City','Latitude','Longitude']]

In [53]:
city_venues_final = countries_df.merge(city_venues_sorted, on='City')

In [54]:
#city_veneus_final.head()

In [55]:
# set number of clusters
kclusters = 10

#city_venues_grouped_clustering

cvgc_df = city_venues_grouped.iloc[:,1:]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cvgc_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([9, 0, 9, 2, 7, 6, 3, 7, 1, 3], dtype=int32)

In [56]:
city_venues_merged = city_venues_final.iloc[:,:2]

In [57]:
afrimain_df3

,Country,City,Type,Latitude,Longitude
0,Algeria,Algiers,CC,28,2.99998
1,Algeria,Oran,MC,35.7033,-0.649298
2,Algeria,Constantine,MC,36.3645,6.60826
3,Algeria,Batna,MC,35.5544,6.17675
4,Algeria,Bab Ezzouar,MC,36.722,3.18567
5,Algeria,Annaba,MC,36.8982,7.75493
6,Angola,Luanda,CC,-8.82727,13.244
7,Angola,Huambo,MC,-12.7765,15.732
8,Angola,Lobito,MC,-12.3507,13.5464
9,Angola,Benguela,MC,-12.579,13.4037


In [58]:
city_venues_merged = city_venues_merged.merge(afrimain_df3, on='City')

In [59]:
city_venues_merged.rename(columns={'Country_x':'Country'}, inplace=True)

In [60]:
city_venues_merged.drop('Country_y', axis=1,inplace=True)

In [61]:
city_venues_merged['Cluster Labels'] = kmeans.labels_

In [62]:
#add clustering labels

city_venues_merged = city_venues_merged.join(city_venues_sorted.iloc[:,1:])

In [63]:
city_venues_merged[city_venues_merged['Country']=='Mauritius']

,Country,City,Type,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
24,Mauritius,Port Louis,CC,-20.1637,57.5045,8,Hotel,Chinese Restaurant,Café,Resort,Fast Food Restaurant,Shopping Mall,Indian Restaurant,Ice Cream Shop,Beach,Lounge,Seafood Restaurant,Historic Site,Italian Restaurant,Brewery,Japanese Restaurant,Asian Restaurant,Restaurant,Diner,Pizza Place,Coffee Shop
25,Mauritius,Curepipe,MC,-20.3151,57.5211,8,Chinese Restaurant,Café,Indian Restaurant,Shopping Mall,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Pizza Place,Department Store,Irish Pub,Sculpture Garden,Volcano,Food Court,Restaurant,Lounge,Spa,Furniture / Home Store,Garden,Portuguese Restaurant,Movie Theater
26,Mauritius,Quatre Bornes,MC,-20.2643,57.48,8,Chinese Restaurant,Café,Shopping Mall,Indian Restaurant,Diner,Pizza Place,Park,Lounge,Restaurant,Italian Restaurant,Steakhouse,Ice Cream Shop,Gym / Fitness Center,Food Court,Fast Food Restaurant,Grocery Store,Seafood Restaurant,Zoo,Sandwich Place,Athletics & Sports
27,Mauritius,Triolet,MC,-20.0575,57.5522,1,Beach,Resort,Hotel,Seafood Restaurant,Bar,Italian Restaurant,Restaurant,Shopping Mall,Cajun / Creole Restaurant,Nightclub,Café,Miscellaneous Shop,Buffet,Mediterranean Restaurant,French Restaurant,Lounge,Department Store,Indian Restaurant,Fish & Chips Shop,Garden
28,Mauritius,Goodlands,MC,-20.0351,57.6589,1,Beach,Resort,Hotel,Bar,Shopping Mall,Seafood Restaurant,Italian Restaurant,Café,Nightclub,Restaurant,Bistro,Buffet,Fast Food Restaurant,Farmers Market,Indian Restaurant,Cajun / Creole Restaurant,Farm,Movie Theater,Pub,Portuguese Restaurant


In [64]:
city_venues_merged

,Country,City,Type,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Algeria,Algiers,CC,28,2.99998,9,Hotel,Restaurant,Mediterranean Restaurant,Coffee Shop,French Restaurant,Park,Cupcake Shop,Burger Joint,Salad Place,Recreation Center,Plaza,Convenience Store,Indian Restaurant,Department Store,Steakhouse,Lounge,Lighthouse,Lake,Italian Restaurant,Gym / Fitness Center
1,Algeria,Oran,MC,35.7033,-0.649298,0,Hotel,Restaurant,Café,Ice Cream Shop,Boat or Ferry,Sandwich Place,Breakfast Spot,Shopping Mall,Burger Joint,Seafood Restaurant,Scenic Lookout,Indian Restaurant,Tea Room,Eastern European Restaurant,Pizza Place,Construction & Landscaping,Dessert Shop,Pedestrian Plaza,Big Box Store,Middle Eastern Restaurant
2,Algeria,Bab Ezzouar,MC,36.722,3.18567,9,Hotel,Mediterranean Restaurant,Coffee Shop,Restaurant,Lake,Plaza,Recreation Center,Salad Place,Burger Joint,Shopping Mall,Metro Station,Park,French Restaurant,Lounge,Steakhouse,Beach,Indian Restaurant,Cupcake Shop,Airport Terminal,Turkish Restaurant
3,Angola,Luanda,CC,-8.82727,13.244,2,Restaurant,Pizza Place,African Restaurant,Lounge,Hotel,Seafood Restaurant,Portuguese Restaurant,Ice Cream Shop,Italian Restaurant,Bar,Mediterranean Restaurant,Beach,BBQ Joint,Coffee Shop,Convenience Store,Supermarket,Fast Food Restaurant,Dessert Shop,Office,Movie Theater
4,Benin,Cotonou,MC,6.3677,2.42525,7,Resort,Hotel,Restaurant,Shopping Mall,Hotel Bar,Bakery,Plaza,Middle Eastern Restaurant,Beach,Lounge,Music Venue,Tapas Restaurant,French Restaurant,Thai Restaurant,Food Court,Pizza Place,Soccer Stadium,Dessert Shop,Harbor / Marina,Hardware Store
5,Botswana,Gaborone,CC,-24.6553,25.9087,6,Shopping Mall,Hotel,Restaurant,Coffee Shop,Café,Burger Joint,Portuguese Restaurant,Fast Food Restaurant,Bistro,Steakhouse,Miscellaneous Shop,Cocktail Bar,Middle Eastern Restaurant,Seafood Restaurant,Brazilian Restaurant,Shop & Service,Soccer Field,Mexican Restaurant,Food,Hotel Bar
6,Cameroon,Yaounde,CC,3.86899,11.5213,3,Bakery,Hotel,Plaza,Bar,Restaurant,Pizza Place,Lounge,Nightclub,Shopping Mall,Café,Food & Drink Shop,Fast Food Restaurant,Bus Station,Pub,Casino,Tennis Court,Cocktail Bar,Coffee Shop,Comedy Club,Piano Bar
7,Cameroon,Douala,MC,4.05374,9.66444,7,Bakery,Lounge,Restaurant,Hotel,Shopping Mall,Nightclub,Big Box Store,Diner,Pizza Place,French Restaurant,Ice Cream Shop,African Restaurant,Department Store,Food Court,Mediterranean Restaurant,Bed & Breakfast,Pharmacy,Plaza,Pub,Café
8,Cape Verde,Santa Maria,MC,16.5987,-22.905,1,Resort,Hotel,Seafood Restaurant,Restaurant,Hotel Bar,Beach,Beach Bar,Surf Spot,Italian Restaurant,Spa,African Restaurant,Lounge,Portuguese Restaurant,Bistro,Steakhouse,Cocktail Bar,Botanical Garden,English Restaurant,Pub,BBQ Joint
9,Democratic Republic of the Congo,Kinshasa,CC,-4.32171,15.3126,3,Hotel,Café,Restaurant,Fast Food Restaurant,Lounge,Plaza,Resort,Furniture / Home Store,Market,Steakhouse,Cocktail Bar,Grocery Store,Pool,Bar,Italian Restaurant,Diner,Pizza Place,Shopping Mall,Breakfast Spot,Airport


In [65]:
afri_lat = -1.7832
afri_long = 20.5085

# create map
map_clusters = folium.Map(location=[afri_lat, afri_long], zoom_start=3, tiles='Mapbox Bright')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_venues_merged['Latitude'], city_venues_merged['Longitude'], city_venues_merged['City'], city_venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#Add legend for clusters 

# Cluster Analysis: Level 1

In [66]:
cluster_dict={}
for i in range(10):
    cluster_dict['cluster'+str(i+1)] = city_venues_merged.loc[city_venues_merged['Cluster Labels'] == i, city_venues_merged.columns[[0,1,2]+ list(range(6, city_venues_merged.shape[1]))]]
                                                              

In [69]:
#city_venues_merged['Cluster Labels']==9.unique()

## Cluster 1

In [70]:
#You can add lists? But I thought you had to append them, probably not if they are both type list. It's concatenation.
#Just want to see how many times each element appears, and perhaps in what column it appears (Histogram? Frequency) 

#len(cluster2_series.unique())
#to see uniqueness might make more sense to look at them on a row basis. Because naturally some clusters are bigger than others. Yeah. 

In [71]:
cluster_dict['cluster1']

,Country,City,Type,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,Algeria,Oran,MC,Hotel,Restaurant,Café,Ice Cream Shop,Boat or Ferry,Sandwich Place,Breakfast Spot,Shopping Mall,Burger Joint,Seafood Restaurant,Scenic Lookout,Indian Restaurant,Tea Room,Eastern European Restaurant,Pizza Place,Construction & Landscaping,Dessert Shop,Pedestrian Plaza,Big Box Store,Middle Eastern Restaurant
12,Egypt,Cairo,CC,Historic Site,Lounge,Café,Italian Restaurant,Egyptian Restaurant,Theater,Hotel,Middle Eastern Restaurant,Coffee Shop,Pastry Shop,Seafood Restaurant,Performing Arts Venue,Hotel Bar,Kebab Restaurant,Bookstore,Dessert Shop,Falafel Restaurant,Plaza,Art Gallery,Neighborhood
13,Egypt,Alexandria,MC,Café,Seafood Restaurant,Coffee Shop,Ice Cream Shop,Sandwich Place,Middle Eastern Restaurant,Historic Site,Restaurant,Egyptian Restaurant,Syrian Restaurant,Bar,Bakery,Italian Restaurant,Lebanese Restaurant,Plaza,Lounge,Juice Bar,Pizza Place,Hotel,Gym
29,Morocco,Rabat,CC,Café,Historic Site,Restaurant,Hotel,Moroccan Restaurant,French Restaurant,Diner,Coffee Shop,Dessert Shop,Pizza Place,Art Gallery,Falafel Restaurant,Movie Theater,Italian Restaurant,Tapas Restaurant,Department Store,Burger Joint,Scenic Lookout,Plaza,Seafood Restaurant
30,Morocco,Casablanca,MC,Café,Coffee Shop,Hotel,Fast Food Restaurant,French Restaurant,Burger Joint,Pub,Tapas Restaurant,Gastropub,Restaurant,Diner,Lounge,Ice Cream Shop,Mediterranean Restaurant,Seafood Restaurant,Bar,Italian Restaurant,Moroccan Restaurant,Chinese Restaurant,Plaza
33,Morocco,Agadir,MC,Restaurant,Resort,Hotel,Café,Moroccan Restaurant,Coffee Shop,Beach,Seafood Restaurant,Nightclub,Diner,Hotel Bar,Snack Place,Italian Restaurant,Surf Spot,Supermarket,Campground,Department Store,Spanish Restaurant,Lounge,Tapas Restaurant
34,Morocco,Tangiers,MC,Café,Hotel,Moroccan Restaurant,Coffee Shop,Italian Restaurant,Resort,Diner,Restaurant,Ice Cream Shop,Middle Eastern Restaurant,Fast Food Restaurant,Scenic Lookout,Pub,Seafood Restaurant,Shopping Mall,Beach,Hostel,Flea Market,Art Gallery,Cocktail Bar
35,Mozambique,Maputo,CC,Café,Hotel,Restaurant,Plaza,Pizza Place,Fast Food Restaurant,Italian Restaurant,Fried Chicken Joint,Indian Restaurant,Seafood Restaurant,Portuguese Restaurant,Coffee Shop,Park,Bar,Bakery,Museum,African Restaurant,Art Gallery,Lounge,Karaoke Bar
45,South Africa,Cape Town,MC,Coffee Shop,Café,Indian Restaurant,Restaurant,Italian Restaurant,Bar,Burger Joint,French Restaurant,Cocktail Bar,Hostel,Museum,Hotel,Pizza Place,Gastropub,Tapas Restaurant,Breakfast Spot,Seafood Restaurant,Steakhouse,Asian Restaurant,Ice Cream Shop
46,South Africa,Durban,MC,Hotel,Café,Restaurant,Coffee Shop,Indian Restaurant,Grocery Store,Fast Food Restaurant,Seafood Restaurant,Burger Joint,Shopping Mall,Italian Restaurant,Gastropub,Breakfast Spot,Gym,Steakhouse,Beach,Bakery,Theater,Japanese Restaurant,French Restaurant


In [72]:
cluster_series_dict = {}

for i in range(1,11):

    cluster_list = cluster_dict['cluster'+str(i)].iloc[:,3:].values.tolist()

    cluster_list_joined = cluster_list[0]

    for venues in cluster_list[1:]:
        cluster_list_joined = cluster_list_joined + venues
    
    cluster_series_dict['cluster'+str(i)] = pd.Series(cluster_list_joined)

In [73]:
cluster_counts = {}
cluster_dataframes = {}

for series, cluster in zip(list(cluster_series_dict.values()),list(cluster_series_dict.keys())):
    cluster_counts[cluster] = series.value_counts().head(20)
    cluster_dataframes[cluster] = cluster_counts[cluster].to_frame(cluster).reset_index() 

#AncientAf_df = AncientAf_count.to_frame('Ancient Africa').reset_index()

In [74]:
clusters_mrgd_new = cluster_dataframes['cluster1'].merge(cluster_dataframes['cluster2'],on='index',how='outer')

for frame in list(cluster_dataframes.values())[2:]:

    clusters_mrgd_new = clusters_mrgd_new.merge(frame,on='index',how='outer')

In [75]:
clusters_mrgd_new.rename(columns={'index':'Venues'},inplace=True)

In [76]:
clusters_mrgd_new.fillna('NaN',inplace=True)

In [77]:
clusters_mrgd_new[clusters_mrgd_new['cluster1']!='NaN']

,Venues,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9,cluster10
0,Café,14,4,3,3,1,2,4,9,3,4
1,Hotel,14,5,5,3,1,2,5,12,1,6
2,Coffee Shop,13,NaN,3,NaN,1,1,2,4,1,4
3,Restaurant,12,5,5,3,1,2,4,11,3,6
4,Seafood Restaurant,12,5,NaN,NaN,1,NaN,NaN,NaN,2,NaN
5,Italian Restaurant,10,5,2,NaN,1,NaN,2,6,2,3
6,Fast Food Restaurant,8,2,5,3,1,NaN,5,8,3,3
7,Pizza Place,8,NaN,4,3,1,NaN,NaN,7,3,3
8,Bar,7,3,4,3,NaN,NaN,2,8,NaN,NaN
9,Burger Joint,7,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,4


In [78]:
print('Countries in cluster: ', cluster_dict['cluster1']['City'].count())

Countries in cluster:  14


Paying particular attention to venues that might be of interest to tourists

These are all the venues for which cluster 1 has a value. Observations:
1) General offerings
    - Greatest quantity and variety of restaurants, cafes, and hotels.
   
2) Moderately unique offerings (defined as being one of three or four clusters that contain these venues):
    - Restaurants:
        - French
        - Middle Eastern
    - Burger joints
    - Ice cream shops
    - Breakfast spots 
   
3) Highly unique offerings (defined as being one of two or less clusters that contain these venues): 
    - Historic sites
    - Art Galleries 

This cluster would be a cosmopolitan food lovers dream. It also has a lot to offer for people who consider themselves to be refined. It is the only cluster with a dense frequency of art galleries and one of two with a number of historic sites. Additionally, it has old countries like Egypt. Therefore, this will be classified as the Ancient Africa Cluster.    

In [79]:
clusters_mrgd_old = clusters_mrgd_new
clusters_mrgd_new.rename(columns={'cluster1':'Ancient Africa'},inplace=True)

## Cluster 2

In [80]:
cluster_dict['cluster2']

,Country,City,Type,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
8,Cape Verde,Santa Maria,MC,Resort,Hotel,Seafood Restaurant,Restaurant,Hotel Bar,Beach,Beach Bar,Surf Spot,Italian Restaurant,Spa,African Restaurant,Lounge,Portuguese Restaurant,Bistro,Steakhouse,Cocktail Bar,Botanical Garden,English Restaurant,Pub,BBQ Joint
27,Mauritius,Triolet,MC,Beach,Resort,Hotel,Seafood Restaurant,Bar,Italian Restaurant,Restaurant,Shopping Mall,Cajun / Creole Restaurant,Nightclub,Café,Miscellaneous Shop,Buffet,Mediterranean Restaurant,French Restaurant,Lounge,Department Store,Indian Restaurant,Fish & Chips Shop,Garden
28,Mauritius,Goodlands,MC,Beach,Resort,Hotel,Bar,Shopping Mall,Seafood Restaurant,Italian Restaurant,Café,Nightclub,Restaurant,Bistro,Buffet,Fast Food Restaurant,Farmers Market,Indian Restaurant,Cajun / Creole Restaurant,Farm,Movie Theater,Pub,Portuguese Restaurant
43,Seychelles,Victoria,CC,Resort,Beach,Seafood Restaurant,Hotel,Restaurant,Italian Restaurant,Airport,Bar,Indian Restaurant,Café,Spa,Cajun / Creole Restaurant,French Restaurant,Historic Site,Cocktail Bar,Airport Lounge,Sandwich Place,Market,Boat or Ferry,Farmers Market
53,Tunisia,Bizerte,MC,Beer Garden,Beach,Resort,Seafood Restaurant,Diner,Italian Restaurant,Fast Food Restaurant,Café,Hotel,Restaurant,Tea Room,Plaza,Coffee Shop,Creperie,Mediterranean Restaurant,Cupcake Shop,Steakhouse,Pizza Place,Pedestrian Plaza,Toll Plaza


In [81]:
clusters_mrgd_new[clusters_mrgd_new['cluster2']!='NaN']

,Venues,Ancient Africa,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9,cluster10
0,Café,14,4,3,3,1,2,4,9,3,4
1,Hotel,14,5,5,3,1,2,5,12,1,6
3,Restaurant,12,5,5,3,1,2,4,11,3,6
4,Seafood Restaurant,12,5,NaN,NaN,1,NaN,NaN,NaN,2,NaN
5,Italian Restaurant,10,5,2,NaN,1,NaN,2,6,2,3
6,Fast Food Restaurant,8,2,5,3,1,NaN,5,8,3,3
8,Bar,7,3,4,3,NaN,NaN,2,8,NaN,NaN
10,Indian Restaurant,7,3,2,NaN,NaN,NaN,NaN,NaN,3,3
12,Lounge,6,2,NaN,3,NaN,2,2,9,3,6
14,French Restaurant,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


In [82]:
print('Countries in cluster: ', cluster_dict['cluster2']['City'].count())

Countries in cluster:  5


These are all the venues for which cluster 2 has a value. Observations:

1) General offerings: 
    - Nothing of note   
    
2) Moderately unique offerings: 
    - Restaurants:
        - Mediterranean
        - French
    - Cocktail bars
    - Bistros
    - Resorts
    
3) Highly unique offerings: 
    - Steakhouses
    - Nightclubs
    - Spas
    - Restaurants:
        - Portugese
        - Cajun/Creole
    - Beaches

This cluster has a number of unique offerings including the most beaches and resorts. Therefore, this will be classifed as the African Getaway Cluster.   

In [83]:
clusters_mrgd_new.rename(columns={'cluster2':'African Getaway'},inplace=True)

## Cluster 3

In [84]:
cluster_dict['cluster3']

,Country,City,Type,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
3,Angola,Luanda,CC,Restaurant,Pizza Place,African Restaurant,Lounge,Hotel,Seafood Restaurant,Portuguese Restaurant,Ice Cream Shop,Italian Restaurant,Bar,Mediterranean Restaurant,Beach,BBQ Joint,Coffee Shop,Convenience Store,Supermarket,Fast Food Restaurant,Dessert Shop,Office,Movie Theater
16,Ghana,Kumasi,MC,African Restaurant,Fast Food Restaurant,Hotel,Chinese Restaurant,Indian Restaurant,Bar,Hotel Bar,Nightclub,Bus Station,Shopping Mall,Pizza Place,Shop & Service,English Restaurant,Food Court,Burrito Place,Café,Restaurant,Shopping Plaza,Snack Place,Brewery
19,Kenya,Nairobi,CC,Coffee Shop,Café,African Restaurant,Bar,Hotel,Fried Chicken Joint,Fast Food Restaurant,Performing Arts Venue,Italian Restaurant,Lounge,Indian Restaurant,Pool,Garden Center,Clothing Store,Chinese Restaurant,Pub,Movie Theater,Spa,Restaurant,Food
37,Nigeria,Abuja,CC,African Restaurant,Hotel,Pizza Place,Café,Restaurant,Department Store,Movie Theater,Fried Chicken Joint,Chinese Restaurant,Convenience Store,Pharmacy,Bar,Fast Food Restaurant,Arcade,Golf Course,Grocery Store,Snack Place,Shopping Mall,Burger Joint,Hotel Bar
50,Sudan,Omdurman,MC,African Restaurant,Market,Fast Food Restaurant,Pizza Place,Hotel,Ice Cream Shop,Concert Hall,Supermarket,Juice Bar,Coffee Shop,Clothing Store,Restaurant,Bus Stop,Bus Station,Shopping Mall,Accessories Store,Burger Joint,Steakhouse,Electronics Store,Pharmacy


In [85]:
clusters_mrgd_new[clusters_mrgd_new['cluster3']!='NaN']

,Venues,Ancient Africa,African Getaway,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9,cluster10
0,Café,14,4,3,3,1,2,4,9,3,4
1,Hotel,14,5,5,3,1,2,5,12,1,6
2,Coffee Shop,13,NaN,3,NaN,1,1,2,4,1,4
3,Restaurant,12,5,5,3,1,2,4,11,3,6
5,Italian Restaurant,10,5,2,NaN,1,NaN,2,6,2,3
6,Fast Food Restaurant,8,2,5,3,1,NaN,5,8,3,3
7,Pizza Place,8,NaN,4,3,1,NaN,NaN,7,3,3
8,Bar,7,3,4,3,NaN,NaN,2,8,NaN,NaN
9,Burger Joint,7,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,4
10,Indian Restaurant,7,3,2,NaN,NaN,NaN,NaN,NaN,3,3


In [86]:
print('Countries in cluster: ', cluster_dict['cluster3']['City'].count())

Countries in cluster:  5


These are all the venues for which cluster 3 has a value. Observations:

1) General offerings: 
    - Nothing of note  
2) Moderately unique offerings: 
    - Restaurants:
        - Chinese 
        - African 
    - Burger joints
3) Highly unique offerings: 
    - Bus stations
    - Fried chicken joints
    - Clothing stores
    - Movie theaters

What stands out about this cluster is its high number of African restaurants, as well as the fact that it has movie thaters and bus stations. The stations imply that it would be fairly easy to get around even if you don't have a car. And the theaters makes one think that this cluster is quite metropolitan. However, not too western. Therefore, this cluster will be classified as the African Experience cluster. 

In [87]:
clusters_mrgd_new.rename(columns={'cluster3':'African Experience'},inplace=True)

## Cluster 4

In [88]:
cluster_dict['cluster4']

,Country,City,Type,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
6,Cameroon,Yaounde,CC,Bakery,Hotel,Plaza,Bar,Restaurant,Pizza Place,Lounge,Nightclub,Shopping Mall,Café,Food & Drink Shop,Fast Food Restaurant,Bus Station,Pub,Casino,Tennis Court,Cocktail Bar,Coffee Shop,Comedy Club,Piano Bar
9,Democratic Republic of the Congo,Kinshasa,CC,Hotel,Café,Restaurant,Fast Food Restaurant,Lounge,Plaza,Resort,Furniture / Home Store,Market,Steakhouse,Cocktail Bar,Grocery Store,Pool,Bar,Italian Restaurant,Diner,Pizza Place,Shopping Mall,Breakfast Spot,Airport
10,Republic of the Congo,Brazzaville,CC,Hotel,Restaurant,Fast Food Restaurant,Café,Plaza,Airport,Lounge,Diner,Bakery,Resort,Pool,Market,Shopping Mall,Furniture / Home Store,French Restaurant,Bar,Cocktail Bar,Pizza Place,Grocery Store,Supermarket


In [89]:
clusters_mrgd_new[clusters_mrgd_new['cluster4']!='NaN']

,Venues,Ancient Africa,African Getaway,African Experience,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9,cluster10
0,Café,14,4,3,3,1,2,4,9,3,4
1,Hotel,14,5,5,3,1,2,5,12,1,6
3,Restaurant,12,5,5,3,1,2,4,11,3,6
6,Fast Food Restaurant,8,2,5,3,1,NaN,5,8,3,3
7,Pizza Place,8,NaN,4,3,1,NaN,NaN,7,3,3
8,Bar,7,3,4,3,NaN,NaN,2,8,NaN,NaN
12,Lounge,6,2,NaN,3,NaN,2,2,9,3,6
17,Plaza,5,NaN,NaN,3,1,1,NaN,NaN,NaN,NaN
18,Shopping Mall,5,2,3,3,1,NaN,5,13,3,3
19,Diner,4,NaN,NaN,2,1,1,NaN,NaN,2,NaN


In [90]:
print('Countries in cluster: ', cluster_dict['cluster4']['City'].count())

Countries in cluster:  3


These are all the venues for which cluster 4 has a value. Observations:

1) Nothing of note
2) Nothing of note
3) Nothing of note 

All of its venues are general, including its unique offerings. Therefore, this cluster will be omitted as it has nothing distinct to offer. 

In [91]:
clusters_mrgd_new.drop('cluster4', 1, inplace=True)

## Cluster 5

In [92]:
cluster_dict['cluster5']

,Country,City,Type,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
22,Libya,Tripoli,CC,Café,Coffee Shop,Italian Restaurant,Bakery,Fast Food Restaurant,Hotel,Pizza Place,Shopping Mall,Seafood Restaurant,Ice Cream Shop,Restaurant,Plaza,Diner,Soccer Field,Middle Eastern Restaurant,Park,Fried Chicken Joint,Supermarket,Latin American Restaurant,Tea Room


In [93]:
clusters_mrgd_new[clusters_mrgd_new['cluster5']!='NaN']

,Venues,Ancient Africa,African Getaway,African Experience,cluster5,cluster6,cluster7,cluster8,cluster9,cluster10
0,Café,14,4,3,1,2,4,9,3,4
1,Hotel,14,5,5,1,2,5,12,1,6
2,Coffee Shop,13,NaN,3,1,1,2,4,1,4
3,Restaurant,12,5,5,1,2,4,11,3,6
4,Seafood Restaurant,12,5,NaN,1,NaN,NaN,NaN,2,NaN
5,Italian Restaurant,10,5,2,1,NaN,2,6,2,3
6,Fast Food Restaurant,8,2,5,1,NaN,5,8,3,3
7,Pizza Place,8,NaN,4,1,NaN,NaN,7,3,3
11,Ice Cream Shop,6,NaN,2,1,NaN,NaN,5,2,NaN
15,Middle Eastern Restaurant,5,NaN,NaN,1,NaN,2,5,NaN,NaN


In [94]:
print('Countries in cluster: ', cluster_dict['cluster5']['City'].count())

Countries in cluster:  1


These are all the venues for which Cluster 5 has a value. Observations:

1) General offerings:
     - Restaurants:
         1) Seafood 
         2) Italian 
         3) Fast Food
         4) Pizza
         
2) Moderately unique offerings: 
    - Bakeries
    - Diners
    - Middle Eastern Restaurants
    - Ice cream shops
    
3) Highly unique offerings 
    - Latin American Restaurants
    - Tea rooms
    - Fried chicken joints 

The most unique thing about this cluster is that it is the only one that has Latin American restaurants. This suggests that there is some Latin American influence on the culture. The fusion of Latin American and African culture could be quite an interesting thing to experience. This cluster will be classified as the Latin Fusion African Cluster. 

In [95]:
clusters_mrgd_new.rename(columns={'cluster5':'Latin Fusion'},inplace=True)

## Cluster 6

In [96]:
cluster_dict['cluster6']

,Country,City,Type,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
31,Morocco,Fez,MC,Café,Hotel,Moroccan Restaurant,Historic Site,Diner,Coffee Shop,Bed & Breakfast,Restaurant,Resort,Tea Room,Fast Food Restaurant,Lounge,Castle,Spa,Sandwich Place,Molecular Gastronomy Restaurant,Plaza,Pub,Chinese Restaurant,Multiplex
32,Morocco,Marrakech,MC,Hotel,Moroccan Restaurant,Café,Bed & Breakfast,Historic Site,Restaurant,Spa,Cocktail Bar,Lounge,Resort,Bistro,French Restaurant,Museum,Garden,Sushi Restaurant,Boutique,Burger Joint,Snack Place,Tea Room,History Museum


In [97]:
clusters_mrgd_new[clusters_mrgd_new['cluster6']!='NaN']

,Venues,Ancient Africa,African Getaway,African Experience,Latin Fusion,cluster6,cluster7,cluster8,cluster9,cluster10
0,Café,14,4,3,1,2,4,9,3,4
1,Hotel,14,5,5,1,2,5,12,1,6
2,Coffee Shop,13,NaN,3,1,1,2,4,1,4
3,Restaurant,12,5,5,1,2,4,11,3,6
12,Lounge,6,2,NaN,NaN,2,2,9,3,6
16,Historic Site,5,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN
17,Plaza,5,NaN,NaN,1,1,NaN,NaN,NaN,NaN
19,Diner,4,NaN,NaN,1,1,NaN,NaN,2,NaN
20,Resort,NaN,5,NaN,NaN,2,NaN,NaN,NaN,NaN
24,Cocktail Bar,NaN,2,NaN,NaN,1,3,NaN,NaN,NaN


In [98]:
print('Countries in cluster: ', cluster_dict['cluster6']['City'].count())

Countries in cluster:  2


These are the venues for which cluster 6 has a vlaue. Observations:

1) General offerings: 
    - Nothing of note 
    
2) Moderately unique offerings: 
    - French restaurants
    - Resorts
    - Bistros
    - Diners
    
3) Highly unique offerings: 
    - Historic sites
    - Spas
    - Tea rooms
    - Bed & Breakfasts
    - Moroccan restaurants
    - Museums
    - Gardens
    - Pubs
    - Molecular gastronomy restaurants
    - Sandwich places
    - Castles 

The most unique thing about this cluster is that it has castles, molecular gastronomy and moroccan restaurants, and museums among other things. Given that it has the most unique offerings out of all of the clusters we've examined, it will be classified as the One-of-a-kind cluster.   

In [99]:
clusters_mrgd_new.rename(columns={'cluster6':'One-of-a-kind'},inplace=True)

## Cluster 7

In [100]:
cluster_dict['cluster7']

,Country,City,Type,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
5,Botswana,Gaborone,CC,Shopping Mall,Hotel,Restaurant,Coffee Shop,Café,Burger Joint,Portuguese Restaurant,Fast Food Restaurant,Bistro,Steakhouse,Miscellaneous Shop,Cocktail Bar,Middle Eastern Restaurant,Seafood Restaurant,Brazilian Restaurant,Shop & Service,Soccer Field,Mexican Restaurant,Food,Hotel Bar
21,Kenya,Kisumu,MC,Shopping Mall,Hotel,Flea Market,Beach,Bed & Breakfast,Lounge,Boat or Ferry,Soup Place,Food Court,Food,Speakeasy,Lake,Scenic Lookout,Fast Food Restaurant,Nightclub,Cocktail Bar,Coffee Shop,Diner,Bar,Department Store
36,Namibia,Windhoek,CC,Hotel,Shopping Mall,Restaurant,Steakhouse,Italian Restaurant,Portuguese Restaurant,Café,Hostel,Bed & Breakfast,Beer Garden,Bistro,Breakfast Spot,Fast Food Restaurant,German Restaurant,Museum,Chinese Restaurant,Pub,Convenience Store,Tapas Restaurant,Mediterranean Restaurant
55,Zambia,Lusaka,CC,Shopping Mall,Hotel,Restaurant,Lounge,Steakhouse,Café,Seafood Restaurant,Bistro,Italian Restaurant,Fast Food Restaurant,American Restaurant,Bar,Boarding House,Farmers Market,Middle Eastern Restaurant,Big Box Store,Bus Station,Supermarket,Movie Theater,Bed & Breakfast
56,Zimbabwe,Harare,CC,Shopping Mall,Restaurant,Hotel,Café,Convenience Store,Performing Arts Venue,Grocery Store,Golf Course,Cocktail Bar,Fast Food Restaurant,Supermarket,Food Court,Steakhouse,Chinese Restaurant,Liquor Store,Multiplex,Pizza Place,Mediterranean Restaurant,Bookstore,Clothing Store


In [101]:
clusters_mrgd_new[clusters_mrgd_new['cluster7']!='NaN']

,Venues,Ancient Africa,African Getaway,African Experience,Latin Fusion,One-of-a-kind,cluster7,cluster8,cluster9,cluster10
0,Café,14,4,3,1,2,4,9,3,4
1,Hotel,14,5,5,1,2,5,12,1,6
2,Coffee Shop,13,NaN,3,1,1,2,4,1,4
3,Restaurant,12,5,5,1,2,4,11,3,6
5,Italian Restaurant,10,5,2,1,NaN,2,6,2,3
6,Fast Food Restaurant,8,2,5,1,NaN,5,8,3,3
8,Bar,7,3,4,NaN,NaN,2,8,NaN,NaN
12,Lounge,6,2,NaN,NaN,2,2,9,3,6
15,Middle Eastern Restaurant,5,NaN,NaN,1,NaN,2,5,NaN,NaN
18,Shopping Mall,5,2,3,1,NaN,5,13,3,3


In [102]:
print('Countries in cluster: ', cluster_dict['cluster7']['City'].count())

Countries in cluster:  5


These are venues for which cluster 7 has values. Observations:

1) General offerings:
    - Variety of palces to eat 
    
2) Moderately unique offerings: 
    - Restaurants: 
        - Chinese
        - Mediterranean 
        - Middle Eastern
        - Bistros
        - Cocktail bars
        
3) Highly unique offerings:
    - Bed & Breakfasts
    - Steakhouses
    - Portugese restaurants 

This cluster doesnt have any highly unique venues that set it apart. Therefore, it will be omitted. 

In [103]:
clusters_mrgd_new.drop('cluster7',1,inplace=True)

## Cluster 8

In [104]:
cluster_dict['cluster8']

,Country,City,Type,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
4,Benin,Cotonou,MC,Resort,Hotel,Restaurant,Shopping Mall,Hotel Bar,Bakery,Plaza,Middle Eastern Restaurant,Beach,Lounge,Music Venue,Tapas Restaurant,French Restaurant,Thai Restaurant,Food Court,Pizza Place,Soccer Stadium,Dessert Shop,Harbor / Marina,Hardware Store
7,Cameroon,Douala,MC,Bakery,Lounge,Restaurant,Hotel,Shopping Mall,Nightclub,Big Box Store,Diner,Pizza Place,French Restaurant,Ice Cream Shop,African Restaurant,Department Store,Food Court,Mediterranean Restaurant,Bed & Breakfast,Pharmacy,Plaza,Pub,Café
11,Cote D’Ivoire,Abidjan,MC,Ice Cream Shop,African Restaurant,Fast Food Restaurant,Bakery,Italian Restaurant,Nightclub,Restaurant,Shopping Mall,Plaza,Café,Lounge,Coffee Shop,Bus Station,Boutique,Steakhouse,Soccer Field,Middle Eastern Restaurant,Supermarket,Bar,Brewery
15,Ghana,Accra,CC,Hotel,African Restaurant,Bar,Fast Food Restaurant,Nightclub,Restaurant,Indian Restaurant,Lounge,Chinese Restaurant,Pizza Place,Shopping Mall,Thai Restaurant,Grocery Store,Sculpture Garden,Bed & Breakfast,Cocktail Bar,Asian Restaurant,Furniture / Home Store,Market,Snack Place
17,Ghana,Tema,MC,Shopping Mall,Fast Food Restaurant,Convenience Store,Golf Course,Fried Chicken Joint,Restaurant,Hotel,African Restaurant,American Restaurant,Breakfast Spot,Mediterranean Restaurant,Food Truck,Food Court,Food,Café,Flea Market,Casino,Fish Market,Airport Food Court,Chinese Restaurant
18,Ghana,Teshi,MC,Hotel,African Restaurant,Fast Food Restaurant,Shopping Mall,Café,Restaurant,American Restaurant,Cocktail Bar,Beach,Bakery,Burger Joint,Chinese Restaurant,Coffee Shop,Bar,Thai Restaurant,Nightclub,Middle Eastern Restaurant,Ice Cream Shop,Lounge,Furniture / Home Store
20,Kenya,Mombasa,MC,Café,Nightclub,Resort,African Restaurant,Shopping Mall,Lounge,Coffee Shop,Pizza Place,Beach,Hotel,Department Store,Italian Restaurant,Burger Joint,Scenic Lookout,Restaurant,Hotel Bar,Airport Terminal,Fast Food Restaurant,Seafood Restaurant,Bar
38,Nigeria,Kano,MC,Shopping Mall,Convenience Store,Middle Eastern Restaurant,Department Store,BBQ Joint,Airport,Hotel,Fried Chicken Joint,Gym / Fitness Center,Mobile Phone Shop,Movie Theater,Bakery,Pizza Place,Resort,Restaurant,Café,Shop & Service,Soccer Field,Nightclub,Indian Restaurant
39,Nigeria,Port Harcourt,MC,Multiplex,Department Store,Shopping Mall,African Restaurant,Hotel,Fast Food Restaurant,Mediterranean Restaurant,Boutique,Bar,Bed & Breakfast,Sports Bar,Movie Theater,Food Court,Karaoke Bar,Clothing Store,Nightclub,Dive Bar,Thai Restaurant,Deli / Bodega,Jewelry Store
40,Nigeria,Lagos,MC,Lounge,Hotel,African Restaurant,Café,Shopping Mall,Italian Restaurant,Pizza Place,Art Gallery,Market,Japanese Restaurant,Indian Restaurant,Bar,Mediterranean Restaurant,Park,Department Store,Nightclub,Restaurant,Diner,Ice Cream Shop,Gym / Fitness Center


In [105]:
clusters_mrgd_new[clusters_mrgd_new['cluster8']!='NaN']

,Venues,Ancient Africa,African Getaway,African Experience,Latin Fusion,One-of-a-kind,cluster8,cluster9,cluster10
0,Café,14,4,3,1,2,9,3,4
1,Hotel,14,5,5,1,2,12,1,6
2,Coffee Shop,13,NaN,3,1,1,4,1,4
3,Restaurant,12,5,5,1,2,11,3,6
5,Italian Restaurant,10,5,2,1,NaN,6,2,3
6,Fast Food Restaurant,8,2,5,1,NaN,8,3,3
7,Pizza Place,8,NaN,4,1,NaN,7,3,3
8,Bar,7,3,4,NaN,NaN,8,NaN,NaN
11,Ice Cream Shop,6,NaN,2,1,NaN,5,2,NaN
12,Lounge,6,2,NaN,NaN,2,9,3,6


In [106]:
print('Countries in cluster: ', cluster_dict['cluster8']['City'].count())

Countries in cluster:  13


These are the venues for which cluster 8 has a vlaue. Observations:

1) General offerings: 
    - The highest occurence of shopping malls by a significant margin
    - The highest occurence of lounges 
    - The second highest occurence of cafe and coffee shops

2) Moderately unique offerings: 
    - Ice Cream Shops
    - Restaurants:
        - Middle Eastern
        - African (most) 
    - Bakeries
    
3) Highly unique offerings:
    - Breakfast spots
    - Beaches
    - Night clubs
    - Thai restaurants 
    
This cluster has a lot to offer in terms of opportunities for shopping and night life. Not to mention things to do throughout the day since it has breakfast spots and coffe shops. What's also very unique about it is that it has the most African restaurants by a significant margin, while still having a fair amount of variety when it comes to places to eat. With a breadth of fun, luxurious, and culinary offerings, this cluster will be classified as the All-in-one cluster.   

In [107]:
clusters_mrgd_new.rename(columns={'cluster8':'All-in-one'},inplace=True)

## Cluster 9

In [108]:
cluster_dict['cluster9']

,Country,City,Type,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
24,Mauritius,Port Louis,CC,Hotel,Chinese Restaurant,Café,Resort,Fast Food Restaurant,Shopping Mall,Indian Restaurant,Ice Cream Shop,Beach,Lounge,Seafood Restaurant,Historic Site,Italian Restaurant,Brewery,Japanese Restaurant,Asian Restaurant,Restaurant,Diner,Pizza Place,Coffee Shop
25,Mauritius,Curepipe,MC,Chinese Restaurant,Café,Indian Restaurant,Shopping Mall,Gym / Fitness Center,Grocery Store,Fast Food Restaurant,Pizza Place,Department Store,Irish Pub,Sculpture Garden,Volcano,Food Court,Restaurant,Lounge,Spa,Furniture / Home Store,Garden,Portuguese Restaurant,Movie Theater
26,Mauritius,Quatre Bornes,MC,Chinese Restaurant,Café,Shopping Mall,Indian Restaurant,Diner,Pizza Place,Park,Lounge,Restaurant,Italian Restaurant,Steakhouse,Ice Cream Shop,Gym / Fitness Center,Food Court,Fast Food Restaurant,Grocery Store,Seafood Restaurant,Zoo,Sandwich Place,Athletics & Sports


In [109]:
clusters_mrgd_new[clusters_mrgd_new['cluster9']!='NaN']

,Venues,Ancient Africa,African Getaway,African Experience,Latin Fusion,One-of-a-kind,All-in-one,cluster9,cluster10
0,Café,14,4,3,1,2,9,3,4
1,Hotel,14,5,5,1,2,12,1,6
2,Coffee Shop,13,NaN,3,1,1,4,1,4
3,Restaurant,12,5,5,1,2,11,3,6
4,Seafood Restaurant,12,5,NaN,1,NaN,NaN,2,NaN
5,Italian Restaurant,10,5,2,1,NaN,6,2,3
6,Fast Food Restaurant,8,2,5,1,NaN,8,3,3
7,Pizza Place,8,NaN,4,1,NaN,7,3,3
10,Indian Restaurant,7,3,2,NaN,NaN,NaN,3,3
11,Ice Cream Shop,6,NaN,2,1,NaN,5,2,NaN


In [110]:
print('Countries in cluster: ', cluster_dict['cluster9']['City'].count())

Countries in cluster:  3


These are the venues for which cluster 9 has a vlaue. Observations:

1) General offerings:
    - Restaurants:
        - Seafood 
        - Italian
        - Fast food 
        - Indian 
    - Shopping Malls 
    
2) Moderatley unique offerings:
    - Ice Cream Shops
    - Chinese restaurants  
    - Diner
   
3) Highly unique offerings:
    - Movie theaters 
    - Gardens
    - Volcano
    
The standout characteristics of this cluster are that it has a variety of places to eat, and a volcano. Since no other cluster has a volcano, this makes it worth keeping. As well as the fact that it has gardens. This cluster will be classifeid as the Picturesque African Experience.      

In [111]:
clusters_mrgd_new.rename(columns={'cluster9':'Picturesque'},inplace=True)

### - Afroasiatic: Chinese restaurants  

## Cluster 10

In [112]:
cluster_dict['cluster10']

,Country,City,Type,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Algeria,Algiers,CC,Hotel,Restaurant,Mediterranean Restaurant,Coffee Shop,French Restaurant,Park,Cupcake Shop,Burger Joint,Salad Place,Recreation Center,Plaza,Convenience Store,Indian Restaurant,Department Store,Steakhouse,Lounge,Lighthouse,Lake,Italian Restaurant,Gym / Fitness Center
2,Algeria,Bab Ezzouar,MC,Hotel,Mediterranean Restaurant,Coffee Shop,Restaurant,Lake,Plaza,Recreation Center,Salad Place,Burger Joint,Shopping Mall,Metro Station,Park,French Restaurant,Lounge,Steakhouse,Beach,Indian Restaurant,Cupcake Shop,Airport Terminal,Turkish Restaurant
14,Ethiopia,Addia Ababa,CC,Hotel,Ethiopian Restaurant,Italian Restaurant,Restaurant,Grocery Store,Pizza Place,Nightclub,Burger Joint,Café,Coffee Shop,Convenience Store,Cupcake Shop,Fast Food Restaurant,Spa,Middle Eastern Restaurant,Massage Studio,Lounge,Golf Course,Greek Restaurant,Gym / Fitness Center
23,Madagascar,Antananarivo,CC,Hotel,Shopping Mall,Department Store,Asian Restaurant,Restaurant,Pizza Place,Mediterranean Restaurant,African Restaurant,French Restaurant,Burger Joint,Tea Room,Sandwich Place,Soccer Field,Market,Lounge,Snack Place,Food Court,Salad Place,Café,Fast Food Restaurant
41,Rwanda,Kigali,CC,Hotel,Café,Nightclub,African Restaurant,Soccer Stadium,Italian Restaurant,Lounge,Coffee Shop,Indian Restaurant,Restaurant,Caucasian Restaurant,Pub,Department Store,Mexican Restaurant,Farmers Market,Bar,French Restaurant,Golf Course,Grocery Store,Japanese Restaurant
51,Tanzania,Dar Es Salaam,CC,Hotel,Bus Station,Resort,Restaurant,African Restaurant,Pizza Place,Lounge,Beach,Fast Food Restaurant,Casino,Café,Motel,BBQ Joint,Football Stadium,Middle Eastern Restaurant,Seafood Restaurant,Electronics Store,Bar,Shopping Mall,Asian Restaurant


In [113]:
clusters_mrgd_new[clusters_mrgd_new['cluster10']!='NaN']

,Venues,Ancient Africa,African Getaway,African Experience,Latin Fusion,One-of-a-kind,All-in-one,Picturesque,cluster10
0,Café,14,4,3,1,2,9,3,4
1,Hotel,14,5,5,1,2,12,1,6
2,Coffee Shop,13,NaN,3,1,1,4,1,4
3,Restaurant,12,5,5,1,2,11,3,6
5,Italian Restaurant,10,5,2,1,NaN,6,2,3
6,Fast Food Restaurant,8,2,5,1,NaN,8,3,3
7,Pizza Place,8,NaN,4,1,NaN,7,3,3
9,Burger Joint,7,NaN,2,NaN,NaN,NaN,NaN,4
10,Indian Restaurant,7,3,2,NaN,NaN,NaN,3,3
12,Lounge,6,2,NaN,NaN,2,9,3,6


In [114]:
print('Countries in cluster: ', cluster_dict['cluster10']['City'].count())

Countries in cluster:  6


These are the venues for whcih cluster 10 had a value. Observations:

1) General offerings:
    - Restaurants:
        - Italian
        - Fast fOOD
        - Pizza 
        - Indian
    - Lounge 
    
2) Moderately unique offerings:
    - Restaurants
        - French
        - Mediterranean 
        - African 
        
3) Highly unique offerings:
    - Salad place
    - Cupcake shop
    
The most unique aspects of this cluster are that it appears to have restaurants that specifically serve salads, and cupcake shops. However, these aren't features that would be a big draw for tourists because they aren't substantial enough. For this reason, this cluster will be dropped.   

In [115]:
clusters_mrgd_new.drop('cluster10',1,inplace=True)

### Final Cluster Table 



In [116]:
clusters_mrgd_new

,Venues,Ancient Africa,African Getaway,African Experience,Latin Fusion,One-of-a-kind,All-in-one,Picturesque
0,Café,14,4,3,1,2,9,3
1,Hotel,14,5,5,1,2,12,1
2,Coffee Shop,13,NaN,3,1,1,4,1
3,Restaurant,12,5,5,1,2,11,3
4,Seafood Restaurant,12,5,NaN,1,NaN,NaN,2
5,Italian Restaurant,10,5,2,1,NaN,6,2
6,Fast Food Restaurant,8,2,5,1,NaN,8,3
7,Pizza Place,8,NaN,4,1,NaN,7,3
8,Bar,7,3,4,NaN,NaN,8,NaN
9,Burger Joint,7,NaN,2,NaN,NaN,NaN,NaN


cluster labels:
    0 = Ancient Africa
    1 = African Getaway
    2 = African Experience 
    4 = Latin Fusion 
    5 = One-of-a-kind
    7 = All-in-one
    8 = Picturesque

### Final Africa Map

In [117]:
cluster_dict['cluster1']

,Country,City,Type,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,Algeria,Oran,MC,Hotel,Restaurant,Café,Ice Cream Shop,Boat or Ferry,Sandwich Place,Breakfast Spot,Shopping Mall,Burger Joint,Seafood Restaurant,Scenic Lookout,Indian Restaurant,Tea Room,Eastern European Restaurant,Pizza Place,Construction & Landscaping,Dessert Shop,Pedestrian Plaza,Big Box Store,Middle Eastern Restaurant
12,Egypt,Cairo,CC,Historic Site,Lounge,Café,Italian Restaurant,Egyptian Restaurant,Theater,Hotel,Middle Eastern Restaurant,Coffee Shop,Pastry Shop,Seafood Restaurant,Performing Arts Venue,Hotel Bar,Kebab Restaurant,Bookstore,Dessert Shop,Falafel Restaurant,Plaza,Art Gallery,Neighborhood
13,Egypt,Alexandria,MC,Café,Seafood Restaurant,Coffee Shop,Ice Cream Shop,Sandwich Place,Middle Eastern Restaurant,Historic Site,Restaurant,Egyptian Restaurant,Syrian Restaurant,Bar,Bakery,Italian Restaurant,Lebanese Restaurant,Plaza,Lounge,Juice Bar,Pizza Place,Hotel,Gym
29,Morocco,Rabat,CC,Café,Historic Site,Restaurant,Hotel,Moroccan Restaurant,French Restaurant,Diner,Coffee Shop,Dessert Shop,Pizza Place,Art Gallery,Falafel Restaurant,Movie Theater,Italian Restaurant,Tapas Restaurant,Department Store,Burger Joint,Scenic Lookout,Plaza,Seafood Restaurant
30,Morocco,Casablanca,MC,Café,Coffee Shop,Hotel,Fast Food Restaurant,French Restaurant,Burger Joint,Pub,Tapas Restaurant,Gastropub,Restaurant,Diner,Lounge,Ice Cream Shop,Mediterranean Restaurant,Seafood Restaurant,Bar,Italian Restaurant,Moroccan Restaurant,Chinese Restaurant,Plaza
33,Morocco,Agadir,MC,Restaurant,Resort,Hotel,Café,Moroccan Restaurant,Coffee Shop,Beach,Seafood Restaurant,Nightclub,Diner,Hotel Bar,Snack Place,Italian Restaurant,Surf Spot,Supermarket,Campground,Department Store,Spanish Restaurant,Lounge,Tapas Restaurant
34,Morocco,Tangiers,MC,Café,Hotel,Moroccan Restaurant,Coffee Shop,Italian Restaurant,Resort,Diner,Restaurant,Ice Cream Shop,Middle Eastern Restaurant,Fast Food Restaurant,Scenic Lookout,Pub,Seafood Restaurant,Shopping Mall,Beach,Hostel,Flea Market,Art Gallery,Cocktail Bar
35,Mozambique,Maputo,CC,Café,Hotel,Restaurant,Plaza,Pizza Place,Fast Food Restaurant,Italian Restaurant,Fried Chicken Joint,Indian Restaurant,Seafood Restaurant,Portuguese Restaurant,Coffee Shop,Park,Bar,Bakery,Museum,African Restaurant,Art Gallery,Lounge,Karaoke Bar
45,South Africa,Cape Town,MC,Coffee Shop,Café,Indian Restaurant,Restaurant,Italian Restaurant,Bar,Burger Joint,French Restaurant,Cocktail Bar,Hostel,Museum,Hotel,Pizza Place,Gastropub,Tapas Restaurant,Breakfast Spot,Seafood Restaurant,Steakhouse,Asian Restaurant,Ice Cream Shop
46,South Africa,Durban,MC,Hotel,Café,Restaurant,Coffee Shop,Indian Restaurant,Grocery Store,Fast Food Restaurant,Seafood Restaurant,Burger Joint,Shopping Mall,Italian Restaurant,Gastropub,Breakfast Spot,Gym,Steakhouse,Beach,Bakery,Theater,Japanese Restaurant,French Restaurant


In [118]:
city_venues_merged.head(5)

,Country,City,Type,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Algeria,Algiers,CC,28,2.99998,9,Hotel,Restaurant,Mediterranean Restaurant,Coffee Shop,French Restaurant,Park,Cupcake Shop,Burger Joint,Salad Place,Recreation Center,Plaza,Convenience Store,Indian Restaurant,Department Store,Steakhouse,Lounge,Lighthouse,Lake,Italian Restaurant,Gym / Fitness Center
1,Algeria,Oran,MC,35.7033,-0.649298,0,Hotel,Restaurant,Café,Ice Cream Shop,Boat or Ferry,Sandwich Place,Breakfast Spot,Shopping Mall,Burger Joint,Seafood Restaurant,Scenic Lookout,Indian Restaurant,Tea Room,Eastern European Restaurant,Pizza Place,Construction & Landscaping,Dessert Shop,Pedestrian Plaza,Big Box Store,Middle Eastern Restaurant
2,Algeria,Bab Ezzouar,MC,36.722,3.18567,9,Hotel,Mediterranean Restaurant,Coffee Shop,Restaurant,Lake,Plaza,Recreation Center,Salad Place,Burger Joint,Shopping Mall,Metro Station,Park,French Restaurant,Lounge,Steakhouse,Beach,Indian Restaurant,Cupcake Shop,Airport Terminal,Turkish Restaurant
3,Angola,Luanda,CC,-8.82727,13.244,2,Restaurant,Pizza Place,African Restaurant,Lounge,Hotel,Seafood Restaurant,Portuguese Restaurant,Ice Cream Shop,Italian Restaurant,Bar,Mediterranean Restaurant,Beach,BBQ Joint,Coffee Shop,Convenience Store,Supermarket,Fast Food Restaurant,Dessert Shop,Office,Movie Theater
4,Benin,Cotonou,MC,6.3677,2.42525,7,Resort,Hotel,Restaurant,Shopping Mall,Hotel Bar,Bakery,Plaza,Middle Eastern Restaurant,Beach,Lounge,Music Venue,Tapas Restaurant,French Restaurant,Thai Restaurant,Food Court,Pizza Place,Soccer Stadium,Dessert Shop,Harbor / Marina,Hardware Store


In [119]:
y = city_venues_merged['Cluster Labels']

In [120]:
city_venues_merged

,Country,City,Type,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Algeria,Algiers,CC,28,2.99998,9,Hotel,Restaurant,Mediterranean Restaurant,Coffee Shop,French Restaurant,Park,Cupcake Shop,Burger Joint,Salad Place,Recreation Center,Plaza,Convenience Store,Indian Restaurant,Department Store,Steakhouse,Lounge,Lighthouse,Lake,Italian Restaurant,Gym / Fitness Center
1,Algeria,Oran,MC,35.7033,-0.649298,0,Hotel,Restaurant,Café,Ice Cream Shop,Boat or Ferry,Sandwich Place,Breakfast Spot,Shopping Mall,Burger Joint,Seafood Restaurant,Scenic Lookout,Indian Restaurant,Tea Room,Eastern European Restaurant,Pizza Place,Construction & Landscaping,Dessert Shop,Pedestrian Plaza,Big Box Store,Middle Eastern Restaurant
2,Algeria,Bab Ezzouar,MC,36.722,3.18567,9,Hotel,Mediterranean Restaurant,Coffee Shop,Restaurant,Lake,Plaza,Recreation Center,Salad Place,Burger Joint,Shopping Mall,Metro Station,Park,French Restaurant,Lounge,Steakhouse,Beach,Indian Restaurant,Cupcake Shop,Airport Terminal,Turkish Restaurant
3,Angola,Luanda,CC,-8.82727,13.244,2,Restaurant,Pizza Place,African Restaurant,Lounge,Hotel,Seafood Restaurant,Portuguese Restaurant,Ice Cream Shop,Italian Restaurant,Bar,Mediterranean Restaurant,Beach,BBQ Joint,Coffee Shop,Convenience Store,Supermarket,Fast Food Restaurant,Dessert Shop,Office,Movie Theater
4,Benin,Cotonou,MC,6.3677,2.42525,7,Resort,Hotel,Restaurant,Shopping Mall,Hotel Bar,Bakery,Plaza,Middle Eastern Restaurant,Beach,Lounge,Music Venue,Tapas Restaurant,French Restaurant,Thai Restaurant,Food Court,Pizza Place,Soccer Stadium,Dessert Shop,Harbor / Marina,Hardware Store
5,Botswana,Gaborone,CC,-24.6553,25.9087,6,Shopping Mall,Hotel,Restaurant,Coffee Shop,Café,Burger Joint,Portuguese Restaurant,Fast Food Restaurant,Bistro,Steakhouse,Miscellaneous Shop,Cocktail Bar,Middle Eastern Restaurant,Seafood Restaurant,Brazilian Restaurant,Shop & Service,Soccer Field,Mexican Restaurant,Food,Hotel Bar
6,Cameroon,Yaounde,CC,3.86899,11.5213,3,Bakery,Hotel,Plaza,Bar,Restaurant,Pizza Place,Lounge,Nightclub,Shopping Mall,Café,Food & Drink Shop,Fast Food Restaurant,Bus Station,Pub,Casino,Tennis Court,Cocktail Bar,Coffee Shop,Comedy Club,Piano Bar
7,Cameroon,Douala,MC,4.05374,9.66444,7,Bakery,Lounge,Restaurant,Hotel,Shopping Mall,Nightclub,Big Box Store,Diner,Pizza Place,French Restaurant,Ice Cream Shop,African Restaurant,Department Store,Food Court,Mediterranean Restaurant,Bed & Breakfast,Pharmacy,Plaza,Pub,Café
8,Cape Verde,Santa Maria,MC,16.5987,-22.905,1,Resort,Hotel,Seafood Restaurant,Restaurant,Hotel Bar,Beach,Beach Bar,Surf Spot,Italian Restaurant,Spa,African Restaurant,Lounge,Portuguese Restaurant,Bistro,Steakhouse,Cocktail Bar,Botanical Garden,English Restaurant,Pub,BBQ Joint
9,Democratic Republic of the Congo,Kinshasa,CC,-4.32171,15.3126,3,Hotel,Café,Restaurant,Fast Food Restaurant,Lounge,Plaza,Resort,Furniture / Home Store,Market,Steakhouse,Cocktail Bar,Grocery Store,Pool,Bar,Italian Restaurant,Diner,Pizza Place,Shopping Mall,Breakfast Spot,Airport


In [121]:
city_venues_final = city_venues_merged[(y!=3)&(y!=6)&(y!=9)]

In [122]:
n = city_venues_final['Cluster Labels'] 

In [123]:
city_venues_final

,Country,City,Type,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
1,Algeria,Oran,MC,35.7033,-0.649298,0,Hotel,Restaurant,Café,Ice Cream Shop,Boat or Ferry,Sandwich Place,Breakfast Spot,Shopping Mall,Burger Joint,Seafood Restaurant,Scenic Lookout,Indian Restaurant,Tea Room,Eastern European Restaurant,Pizza Place,Construction & Landscaping,Dessert Shop,Pedestrian Plaza,Big Box Store,Middle Eastern Restaurant
3,Angola,Luanda,CC,-8.82727,13.244,2,Restaurant,Pizza Place,African Restaurant,Lounge,Hotel,Seafood Restaurant,Portuguese Restaurant,Ice Cream Shop,Italian Restaurant,Bar,Mediterranean Restaurant,Beach,BBQ Joint,Coffee Shop,Convenience Store,Supermarket,Fast Food Restaurant,Dessert Shop,Office,Movie Theater
4,Benin,Cotonou,MC,6.3677,2.42525,7,Resort,Hotel,Restaurant,Shopping Mall,Hotel Bar,Bakery,Plaza,Middle Eastern Restaurant,Beach,Lounge,Music Venue,Tapas Restaurant,French Restaurant,Thai Restaurant,Food Court,Pizza Place,Soccer Stadium,Dessert Shop,Harbor / Marina,Hardware Store
7,Cameroon,Douala,MC,4.05374,9.66444,7,Bakery,Lounge,Restaurant,Hotel,Shopping Mall,Nightclub,Big Box Store,Diner,Pizza Place,French Restaurant,Ice Cream Shop,African Restaurant,Department Store,Food Court,Mediterranean Restaurant,Bed & Breakfast,Pharmacy,Plaza,Pub,Café
8,Cape Verde,Santa Maria,MC,16.5987,-22.905,1,Resort,Hotel,Seafood Restaurant,Restaurant,Hotel Bar,Beach,Beach Bar,Surf Spot,Italian Restaurant,Spa,African Restaurant,Lounge,Portuguese Restaurant,Bistro,Steakhouse,Cocktail Bar,Botanical Garden,English Restaurant,Pub,BBQ Joint
11,Cote D’Ivoire,Abidjan,MC,5.40912,-4.04221,7,Ice Cream Shop,African Restaurant,Fast Food Restaurant,Bakery,Italian Restaurant,Nightclub,Restaurant,Shopping Mall,Plaza,Café,Lounge,Coffee Shop,Bus Station,Boutique,Steakhouse,Soccer Field,Middle Eastern Restaurant,Supermarket,Bar,Brewery
12,Egypt,Cairo,CC,30.0488,31.2437,0,Historic Site,Lounge,Café,Italian Restaurant,Egyptian Restaurant,Theater,Hotel,Middle Eastern Restaurant,Coffee Shop,Pastry Shop,Seafood Restaurant,Performing Arts Venue,Hotel Bar,Kebab Restaurant,Bookstore,Dessert Shop,Falafel Restaurant,Plaza,Art Gallery,Neighborhood
13,Egypt,Alexandria,MC,31.199,29.8944,0,Café,Seafood Restaurant,Coffee Shop,Ice Cream Shop,Sandwich Place,Middle Eastern Restaurant,Historic Site,Restaurant,Egyptian Restaurant,Syrian Restaurant,Bar,Bakery,Italian Restaurant,Lebanese Restaurant,Plaza,Lounge,Juice Bar,Pizza Place,Hotel,Gym
15,Ghana,Accra,CC,5.56001,-0.205744,7,Hotel,African Restaurant,Bar,Fast Food Restaurant,Nightclub,Restaurant,Indian Restaurant,Lounge,Chinese Restaurant,Pizza Place,Shopping Mall,Thai Restaurant,Grocery Store,Sculpture Garden,Bed & Breakfast,Cocktail Bar,Asian Restaurant,Furniture / Home Store,Market,Snack Place
16,Ghana,Kumasi,MC,6.69808,-1.62304,2,African Restaurant,Fast Food Restaurant,Hotel,Chinese Restaurant,Indian Restaurant,Bar,Hotel Bar,Nightclub,Bus Station,Shopping Mall,Pizza Place,Shop & Service,English Restaurant,Food Court,Burrito Place,Café,Restaurant,Shopping Plaza,Snack Place,Brewery


cluster labels:
    0 = Ancient Africa
    1 = African Getaway
    2 = African Experience 
    4 = Latin Fusion 
    5 = One-of-a-kind
    7 = All-in-one
    8 = Picturesque

In [124]:
city_venues_final_part2 = city_venues_final.iloc[:,6:] 

In [125]:
city_venues_final = city_venues_final.iloc[:,:6].reset_index(drop=True)

In [126]:
city_venues_final['New Cluster Label'] = None

In [127]:
for d,label in enumerate(city_venues_final['Cluster Labels']):  

    if city_venues_final['Cluster Labels'][d] == 0:
        city_venues_final['New Cluster Label'][d] = 'Ancient Africa'
    elif city_venues_final['Cluster Labels'][d]==1:
        city_venues_final['New Cluster Label'][d] = 'African Getaway'
    elif city_venues_final['Cluster Labels'][d]==2:
        city_venues_final['New Cluster Label'][d] = 'African Experience'
    elif city_venues_final['Cluster Labels'][d]==4:
        city_venues_final['New Cluster Label'][d] = 'Latin Fusion'
    elif city_venues_final['Cluster Labels'][d]==5:
        city_venues_final['New Cluster Label'][d] = 'One-of-a-kind'
    elif city_venues_final['Cluster Labels'][d]==7:
        city_venues_final['New Cluster Label'][d] = 'All-in-one'
    else: 
        city_venues_final['New Cluster Label'][d] = 'Picturesque'


/Users/mbongeni/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/mbongeni/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/mbongeni/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/mbongeni/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.p

In [128]:
city_venues_final.head()

,Country,City,Type,Latitude,Longitude,Cluster Labels,New Cluster Label
0,Algeria,Oran,MC,35.7033,-0.649298,0,Ancient Africa
1,Angola,Luanda,CC,-8.82727,13.244,2,African Experience
2,Benin,Cotonou,MC,6.3677,2.42525,7,All-in-one
3,Cameroon,Douala,MC,4.05374,9.66444,7,All-in-one
4,Cape Verde,Santa Maria,MC,16.5987,-22.905,1,African Getaway


In [129]:
city_venues_final_part2.reset_index(drop=True,inplace=True)

In [130]:
city_venues_final.join(city_venues_final_part2)

,Country,City,Type,Latitude,Longitude,Cluster Labels,New Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Algeria,Oran,MC,35.7033,-0.649298,0,Ancient Africa,Hotel,Restaurant,Café,Ice Cream Shop,Boat or Ferry,Sandwich Place,Breakfast Spot,Shopping Mall,Burger Joint,Seafood Restaurant,Scenic Lookout,Indian Restaurant,Tea Room,Eastern European Restaurant,Pizza Place,Construction & Landscaping,Dessert Shop,Pedestrian Plaza,Big Box Store,Middle Eastern Restaurant
1,Angola,Luanda,CC,-8.82727,13.244,2,African Experience,Restaurant,Pizza Place,African Restaurant,Lounge,Hotel,Seafood Restaurant,Portuguese Restaurant,Ice Cream Shop,Italian Restaurant,Bar,Mediterranean Restaurant,Beach,BBQ Joint,Coffee Shop,Convenience Store,Supermarket,Fast Food Restaurant,Dessert Shop,Office,Movie Theater
2,Benin,Cotonou,MC,6.3677,2.42525,7,All-in-one,Resort,Hotel,Restaurant,Shopping Mall,Hotel Bar,Bakery,Plaza,Middle Eastern Restaurant,Beach,Lounge,Music Venue,Tapas Restaurant,French Restaurant,Thai Restaurant,Food Court,Pizza Place,Soccer Stadium,Dessert Shop,Harbor / Marina,Hardware Store
3,Cameroon,Douala,MC,4.05374,9.66444,7,All-in-one,Bakery,Lounge,Restaurant,Hotel,Shopping Mall,Nightclub,Big Box Store,Diner,Pizza Place,French Restaurant,Ice Cream Shop,African Restaurant,Department Store,Food Court,Mediterranean Restaurant,Bed & Breakfast,Pharmacy,Plaza,Pub,Café
4,Cape Verde,Santa Maria,MC,16.5987,-22.905,1,African Getaway,Resort,Hotel,Seafood Restaurant,Restaurant,Hotel Bar,Beach,Beach Bar,Surf Spot,Italian Restaurant,Spa,African Restaurant,Lounge,Portuguese Restaurant,Bistro,Steakhouse,Cocktail Bar,Botanical Garden,English Restaurant,Pub,BBQ Joint
5,Cote D’Ivoire,Abidjan,MC,5.40912,-4.04221,7,All-in-one,Ice Cream Shop,African Restaurant,Fast Food Restaurant,Bakery,Italian Restaurant,Nightclub,Restaurant,Shopping Mall,Plaza,Café,Lounge,Coffee Shop,Bus Station,Boutique,Steakhouse,Soccer Field,Middle Eastern Restaurant,Supermarket,Bar,Brewery
6,Egypt,Cairo,CC,30.0488,31.2437,0,Ancient Africa,Historic Site,Lounge,Café,Italian Restaurant,Egyptian Restaurant,Theater,Hotel,Middle Eastern Restaurant,Coffee Shop,Pastry Shop,Seafood Restaurant,Performing Arts Venue,Hotel Bar,Kebab Restaurant,Bookstore,Dessert Shop,Falafel Restaurant,Plaza,Art Gallery,Neighborhood
7,Egypt,Alexandria,MC,31.199,29.8944,0,Ancient Africa,Café,Seafood Restaurant,Coffee Shop,Ice Cream Shop,Sandwich Place,Middle Eastern Restaurant,Historic Site,Restaurant,Egyptian Restaurant,Syrian Restaurant,Bar,Bakery,Italian Restaurant,Lebanese Restaurant,Plaza,Lounge,Juice Bar,Pizza Place,Hotel,Gym
8,Ghana,Accra,CC,5.56001,-0.205744,7,All-in-one,Hotel,African Restaurant,Bar,Fast Food Restaurant,Nightclub,Restaurant,Indian Restaurant,Lounge,Chinese Restaurant,Pizza Place,Shopping Mall,Thai Restaurant,Grocery Store,Sculpture Garden,Bed & Breakfast,Cocktail Bar,Asian Restaurant,Furniture / Home Store,Market,Snack Place
9,Ghana,Kumasi,MC,6.69808,-1.62304,2,African Experience,African Restaurant,Fast Food Restaurant,Hotel,Chinese Restaurant,Indian Restaurant,Bar,Hotel Bar,Nightclub,Bus Station,Shopping Mall,Pizza Place,Shop & Service,English Restaurant,Food Court,Burrito Place,Café,Restaurant,Shopping Plaza,Snack Place,Brewery


In [131]:
rainbow2

NameError: name 'rainbow2' is not defined

In [132]:
for f,cluster in enumerate(city_venues_final['Cluster Labels']):  
    if city_venues_final['Cluster Labels'][f] == 8:
        city_venues_final['Cluster Labels'][f] = 6
    else:
        pass

/Users/mbongeni/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [133]:
city_venues_final['Cluster Labels'].unique()

array([0, 2, 7, 1, 4, 6, 5])

In [143]:
afri_lat = -1.7832
afri_long = 20.5085

# create map
map_clusters_fin = folium.Map(location=[afri_lat, afri_long], zoom_start=3, tiles='Mapbox Bright')

# set color scheme for the clusters
x2 = np.arange(kclusters)
ys2 = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array2 = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow2 = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lon, poi, cluster, label in zip(city_venues_final['Latitude'], city_venues_final['Longitude'], city_venues_final['City'], city_venues_final['Cluster Labels'], city_venues_final['New Cluster Label']):
    label2 = folium.Popup(str(poi) + ', ' + label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label2,
        color=rainbow2[cluster],
        fill=True,
        fill_color=rainbow2[cluster],
        fill_opacity=0.7).add_to(map_clusters_fin)
    #print(cluster-1)
      
#map_clusters_fin

#Add legend for clusters 

In [144]:
#Add legend for clusters 

from folium.plugins import FloatImage

image_file = '/Users/mbongeni/Documents/Career/Data Science/Applied Data Science/4_Capstone/Final Assignment/Legend3.png'

FloatImage(image_file, bottom=0,left=71).add_to(map_clusters_fin)

map_clusters_fin.save('FinalClustersMap.html')

#map_clusters_fin

In [145]:
map_clusters_fin

In [ ]:
os.path

In [ ]:
rainbow2

In [ ]:
for row in range(clusters_mrgd_fill.shape[0]): 
    x = clusters_mrgd_fill.loc[row].values
    if np.count_nonzero(x=='NaN') >= 7:
        pass 
        #put in data frame 
    else:
        clusters_mrgd_fill.drop([row],inplace=True)
        #drop it from dataframe

In [ ]:
#Migrating Dar Es Salam to Alt African Experience Cluster 
tan_df = cluster_dict['cluster10'][cluster_dict['cluster10']['Country']=='Tanzania']
AltAfExp = pd.concat([cluster_dict['cluster3'],tan_df])
AltAfExp.value_counts().head(20)

I migrated Dar Es Salam because it had African Restaurants in common with the AltAfExp cluster and it also had a 
beach in common with Angola, which was already in that cluster. These two factors, plus the fact that Dar Es Salam
is a popular city made me decide to move it from the 10th cluster which I decided to drop. 


In [ ]:
AltAfExp_list = AltAfExp.iloc[:,3:].values.tolist()

AltAfExp_list_joined = AltAfExp_list[0]

for venues in AltAfExp_list[1:]:
    AltAfExp_list_joined = AltAfExp_list_joined + venues

AltAfExp_series = pd.Series(AltAfExp_list_joined)

In [ ]:
AltAfExp_count = AltAfExp_series.value_counts().head(20)

In [ ]:
#0 means that there's 0 of that venue amongst that clusters top venues
#and a number means that it appears however many times amongst the top venues of that cluster 

## Lesson
- The issue was that there were spaces in how the cities in afriman_df1 were spelt. So they could not be seen as exactly equivalent. Which is why it worked when in was used instead of ==. 
- The lesson is to always check if things are the same and think of spaces as characters 

afrimain_df2 = pd.merge(afrimain_df1, timezones_df1, on='City', how='left')

timezones_df1[timezones_df1['Country']=='Cameroon']

## Lesson

- I'm learining a patience that I think is necessary for data science 
- And data with special characters needs to be read in in a certain way 

afrimain_df2['City'][34]

afrimain_df1['Country_y'] = np.nan
for b, capital in enumerate(timezones_df1['City']):
    #print(capital)
    for r, city in enumerate(afrimain_df1['City']):  
        if capital in city:
            afrimain_df1['Country_y'][r] = timezones_df1['Country'][b]
        else:
            pass
        
        #print(capital == city)

afrimain_df1

## Lesson
Work with what is present in every entry. Should have noticed that there isn't major city information for every country and therefore there wouldn't be country information for every capital doing it the way you had decided: Extracting country information connected to major city information. 

afrimain_df1

afri_df1